In [121]:
# Prepare graph data...
import numpy as np
import matplotlib.pyplot as plt
import os
from collections import namedtuple
Graph = namedtuple('Graph', ['X', 'Ri', 'Ro', 'y'])
class GraphDataset():
    def __init__(self, input_dir, n_samples=None):
        input_dir = os.path.expandvars(input_dir)
        filenames = [os.path.join(input_dir, f) for f in os.listdir(input_dir)
                     if f.startswith('event') and f.endswith('.npz')]
        self.filenames = (
            filenames[:n_samples] if n_samples is not None else filenames)

    def __getitem__(self, index):
        return load_graph(self.filenames[index])

    def __len__(self):
        return len(self.filenames)
def get_dataset(input_dir,n_files):
    return GraphDataset(input_dir, n_files)
def load_graph(filename):
    graph_dir = os.path.join(os.getcwd(), 'graphs')
    # Construct the full path to the specified file
    full_path = os.path.join(graph_dir, filename)
    """Read a single graph NPZ"""
    with np.load(full_path) as f:
        return sparse_to_graph(**dict(f.items()))
def sparse_to_graph(X, Ri_rows, Ri_cols, Ro_rows, Ro_cols, y, dtype=np.float32):
    n_nodes, n_edges = X.shape[0], Ri_rows.shape[0]
    Ri = np.zeros((n_nodes, n_edges), dtype=dtype)
    Ro = np.zeros((n_nodes, n_edges), dtype=dtype)
    Ri[Ri_rows, Ri_cols] = 1
    Ro[Ro_rows, Ro_cols] = 1
    return Graph(X, Ri, Ro, y)
#Function to load X, Ri, Ro, Y.
def load_raw(graph_name,xyr):
    graph_ex=load_graph(graph_name)
    #Load raw data
    y=graph_ex.y
    Ri=graph_ex.Ri
    Ro=graph_ex.Ro
    X=graph_ex.X
    if xyr=='X':
        return X
    elif xyr=='Ri':
        return Ri
    elif xyr=='Ro':
        return Ro
    else:
        return y
# Load X data of dimension Nv x 4(including color)
import os
import pandas as pd
dir_ = os.path.join(os.getcwd(), 'color')
#dir_ = os.path.join(os.getcwd(), 'coloredX')
file_path = os.path.join(dir_, "event000001000_g000.csv")
v = pd.read_csv(file_path)
print("features data loaded...")
#Convenient representation of edge data 
#edges = [[i1,j1], [i2,j2], ... ]; i1, i2,... are outgoing-nodes, and j1, j2, ... are incoming-nodes
import json
graph='event000001000_g000'
with open("./networks/"+graph+".json", "r") as json_file:
        _,_,edges= json.load(json_file)
print("edge data loaded...")

features data loaded...
edge data loaded...


In [122]:
g_name='event000001000_g000.npz'
Ri=load_raw(g_name,'Ri')
Ro=load_raw(g_name,'Ro')
Ri,Ro=Ri.T,Ro.T
Rio=Ri+Ro
Rio.shape
y=load_raw(g_name,'y')

In [123]:
import json
# Specify the path to your JSON file
json_file_path = 'event000001000_g000_pairs.json'
# Open the file and load the JSON data
with open(json_file_path, 'r') as file:
    data = json.load(file)
# Access the data
all_pairs=data["all_pairs"]
real_pairs,fake_pairs=data["real_pairs"],data["fake_pairs"]
quads,f_quads=data["quads"],data["f_quads"]

In [124]:
quads[0]

[[0, 1], [1, 2], [2, 3]]

In [148]:
X=v[0:4].values

In [126]:
import pennylane as qml
from qiskit.circuit import Parameter
# Create a quantum circuit with Nv qubits
Nv = 4
num_qubits = Nv * 4  # 4 qubits per row (x,y,z,color)
#layers=4
# Define a list of trainable parameters for the entanglement
num_params = Nv  # Number of trainable parameters for the entanglement
import torch
import torch.nn as nn
import torch_geometric.nn as geom_nn
import pennylane as qml
from pennylane import numpy as np
ansatz = torch.randn(Nv, requires_grad=True)
#ansatz = np.random.random(size=(num_params*layers))

In [127]:
def quantum_circuit(X,ansatz):
    for i in range(Nv):
        for j in range(4):
            if j==0:
                qml.RX(X[i*4+j], wires = i * 4 + j)
            elif j==1:
                qml.RY(X[i*4+j], wires = i * 4 + j)
            else:
                qml.RZ(X[i*4+j], wires = i * 4 + j)
        #Entangle 3 positions
        qml.CNOT(wires = [i * 4, i * 4 + 1])
        qml.CNOT(wires = [i * 4 + 1, i * 4 + 2])
        qml.CNOT(wires = [i * 4 + 2, i * 4])
        #Entangle color and position-x.
        qml.CNOT(wires = [i * 4 + 3, i * 4])
        # Encode each feature value into a qubit 
    # Apply trainable entanglement gates
    for i in range(Nv):
        qml.RZ(ansatz[i], wires = i * 4)
    #Fully connect
    for i in range(Nv):
        for j in range(i + 1, Nv):
            # Use trainable parameters for entanglement angles
            qml.CNOT(wires = [i * 4, j * 4])
# Updated quantum_layer function
@qml.qnode(qml.device("default.qubit", wires=4*Nv))
def quantum_layer(x,params):
    quantum_circuit(x,params)
    return [qml.expval(qml.PauliZ(i*4)) for i in range(Nv)]
# Create a custom quantum layer with PennyLane
class QuantumLayer(nn.Module):
    def __init__(self, num_qubits):
        super(QuantumLayer, self).__init__()
        self.num_qubits = num_qubits
    def forward(self, x, params):
        return quantum_layer(x,params)

In [143]:
class CustomMessagePassingLayer(MessagePassing):
    def __init__(self, in_channels, out_channels):
        super(CustomMessagePassingLayer, self).__init__(aggr='add')
        self.linear = nn.Linear(in_channels * 2, out_channels)  # Input size is doubled for edge pairs
    def forward(self, x, edge_index):
        edge_index, _ = add_self_loops(edge_index, num_nodes=x.size(0))
        return self.propagate(edge_index, x=x)
    def message(self, x_i, x_j):
        # Concatenate node features for each edge
        message = torch.cat([x_i, x_j], dim=1)
        return self.linear(message)
    def update(self, aggr_out, x):
        return aggr_out + x

class ClassicalMessagePassing(nn.Module):
    def __init__(self, num_nodes, num_edges, num_node_features, num_edge_features):
        super(ClassicalMessagePassing, self).__init__()
        
        
        self.edge_classifier = nn.Linear(num_node_features, 1)

    def forward(self, x, edge_index):
        # Classical message passing layer
        classical_output = self.message_passing_layer(x.view(self.num_nodes, -1), edge_index)
        # Temporary edge features by combining node features (e.g., addition)
        temp_edge_features = torch.tensor([])
        for i in range(num_nodes):
            for j in range(i + 1, num_nodes):
                edge_f = classical_output[i] + classical_output[j]
                temp_edge_features = torch.cat((temp_edge_features, edge_f.unsqueeze(0)), dim=0)
        # Edge-level prediction
        edge_scores = self.edge_classifier(temp_edge_features)
        return edge_scores.squeeze()

# Assuming x and edge_index are your input features and edge connections
x = v[0:4].values
new_x = torch.tensor(np.delete(x, -1, axis=1)).to(torch.float32)

classical_model = ClassicalMessagePassing(num_nodes, num_edges, num_node_features, num_edge_features)
classical_output = classical_model(new_x, edge_index)
print(classical_output)

tensor([1.2556, 1.3130, 1.3577, 1.3790, 1.4237, 1.4811],
       grad_fn=<SqueezeBackward0>)


In [172]:
# Define a model that integrates quantum and classical components
class QuantumMessagePassing(nn.Module):
    def __init__(self, num_nodes, num_edges, num_node_features, 
                 num_edge_features, num_qubits, quantum_params):
        super(QuantumMessagePassing, self).__init__()
        self.num_nodes = num_nodes
        self.num_edges = num_edges
        self.num_node_features = num_node_features
        self.num_edge_features = num_edge_features
        self.num_qubits = num_qubits
        self.quantum_params = quantum_params
        self.quantum_layer = QuantumLayer(num_qubits)
        self.message_passing_layer = CustomMessagePassingLayer(num_node_features, num_edge_features)
        self.edge_temp = nn.Linear(num_node_features, 1)
        self.edge_classifier = nn.Linear(num_edge_features, 1)  # Input size is doubled for edge pairs
    def forward(self, x, edge_index):
        new_x = torch.tensor(np.delete(x, -1, axis=1)).to(torch.float32)
        # Classical message passing layer
        classical_output = self.message_passing_layer(new_x.view(self.num_nodes, -1), edge_index)
        # Temporary edge features by combining node features (e.g., addition)
        classical = torch.tensor([])
        for i in range(num_nodes):
            for j in range(i + 1, num_nodes):
                edge_f = classical_output[i] + classical_output[j]
                classical = torch.cat((classical, edge_f.unsqueeze(0)), dim=0)
        classical_edge_score=(self.edge_temp(classical)).view(1, 6)[0]

        # Quantum layer
        new_x_q=x.flatten()
        quantum_output = self.quantum_layer(new_x_q, self.quantum_params)
        
        first_q=torch.stack([quantum_output[n] for n in edge_index[0]])
        second_q=torch.stack([quantum_output[n] for n in edge_index[1]])
        quantum_edge=first_q+second_q
    
        # Combine quantum and classical outputs for each node in the edge pair
        combined_output = quantum_edge+classical_edge_score
        combined_output = combined_output.float()
        combined_output = combined_output.reshape(6, 1)
        # Edge-level prediction
        edge_scores = self.edge_classifier(combined_output)
        return edge_scores.squeeze()

In [173]:
# Example usage
# Example usage
num_nodes = 4
num_edges = 6
num_node_features = 3
num_edge_features = 1  # Scalar edge feature
edge_index = torch.tensor([[0, 0, 0, 1, 1, 2], [1, 2, 3, 2, 3, 3]])
quantum_params = torch.randn(Nv, requires_grad=True)
model = QuantumMessagePassing(num_nodes,num_edges,
                              num_node_features,num_edge_features,
                              num_qubits, quantum_params)

# Assuming x and edge_index are your input features and edge connections
edge_index = torch.tensor([[0, 0, 0, 1, 1, 2], 
                           [1, 2, 3, 2, 3, 3]])  # Fully connected graph

output = model(X, edge_index)

In [174]:
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

In [175]:
# Assuming true_labels is a tensor with true labels for each edge (0 or 1)
true_labels = torch.tensor([1, 0, 0, 1, 0, 1], dtype=torch.float32)
# Number of training epochs
num_epochs = 100
for epoch in range(num_epochs):
    # Forward pass
    edge_scores = model(X, edge_index)
    # Calculate the loss
    loss = criterion(edge_scores, true_labels)
    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward(retain_graph=True)
    optimizer.step()
    # Print loss for monitoring
    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {loss.item()}")

Epoch 1/100, Loss: 0.6827016472816467
Epoch 2/100, Loss: 0.6826961636543274
Epoch 3/100, Loss: 0.6826908588409424
Epoch 4/100, Loss: 0.6826856732368469
Epoch 5/100, Loss: 0.682680606842041
Epoch 6/100, Loss: 0.6826757788658142
Epoch 7/100, Loss: 0.6826708316802979
Epoch 8/100, Loss: 0.6826660633087158
Epoch 9/100, Loss: 0.6826614737510681
Epoch 10/100, Loss: 0.6826568245887756
Epoch 11/100, Loss: 0.6826522946357727
Epoch 12/100, Loss: 0.6826479434967041
Epoch 13/100, Loss: 0.6826435923576355
Epoch 14/100, Loss: 0.6826391816139221
Epoch 15/100, Loss: 0.6826348900794983
Epoch 16/100, Loss: 0.682630717754364
Epoch 17/100, Loss: 0.682626485824585
Epoch 18/100, Loss: 0.6826222538948059
Epoch 19/100, Loss: 0.6826181411743164
Epoch 20/100, Loss: 0.6826140880584717
Epoch 21/100, Loss: 0.682610034942627
Epoch 22/100, Loss: 0.6826059818267822
Epoch 23/100, Loss: 0.6826019883155823
Epoch 24/100, Loss: 0.6825979351997375
Epoch 25/100, Loss: 0.6825940012931824
Epoch 26/100, Loss: 0.6825900673866272